In [1]:
import pandas as pd
import requests
import os


- Mediatree doc https://keywords.mediatree.fr/api/docs/#/paths/~1api~1v2~1subtitle~1/get

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Auth

In [3]:
url_auth = "https://keywords.mediatree.fr/api/auth/token"

In [4]:
mediatree_username = os.environ.get("MEDIATREE_USERNAME")
mediatree_password = os.environ.get("MEDIATREE_PASSWORD")

In [5]:
url_auth2 = f"https://keywords.mediatree.fr/api/auth/token/?username={mediatree_username}&password={mediatree_password}&grant_type=password"
requests.put(url_auth)

<Response [405]>

In [6]:
# curl -X POST https://keywords.mediatree.fr/api/auth/token/ -H "Content-Type: application/x-www-form-urlencoded" -d "grant_type=password" -d "username=XXX" -d "password=XXX"

In [7]:
import requests

# For password grant type
data = {
    "grant_type": "password",
    "username": os.environ.get("MEDIATREE_USERNAME"),
    "password": os.environ.get("MEDIATREE_PASSWORD"),
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url_auth, data=data, headers=headers)

if response.status_code == 200:
    token_data = response.json()
    access_token = token_data['data']['access_token']
    refresh_token = token_data['data']['refresh_token']
    expires_in = token_data['data']['expires_in']
    print(f"Access Token: {access_token}")
    print(f"Refresh Token: {refresh_token}")
    print(f"Expires in: {expires_in} seconds")
else:
    print(f"Authentication failed with status code: {response.status_code}")
    print(response.text)

Authentication failed with status code: 405
{"error": "MethodNotAllowed", "message": "Method not allowed"}


# Get subtitles

In [8]:
channel_sudradio = "sud-radio"
channel_cnews = "itele"

In [9]:
MEDIATREE_TOKEN = os.environ.get("MEDIATREE_TOKEN")

## Get all channels

In [49]:
url_fifo = "https://keywords.mediatree.fr/api/fifo"

In [51]:
params = {
    "token":MEDIATREE_TOKEN,
    "size":30,
}
response = requests.get(url_fifo,params = params)
response.json()

{'total_results': 19,
 'number_pages': 1,
 'data': [{'name': 'rfi',
   'title': 'RFI',
   'radio': True,
   'logo': 'logos/default.png',
   'modified': 1726160419},
  {'name': 'franceinfotv',
   'title': 'France Info:',
   'radio': False,
   'logo': 'logos/default.png',
   'modified': 1726160423},
  {'name': 'lci',
   'title': 'LCI',
   'radio': False,
   'logo': 'logos/default.png',
   'modified': 1726160423},
  {'name': 'fr3-idf',
   'title': 'France 3 Idf',
   'radio': False,
   'logo': 'logos/default.png',
   'modified': 1726160425},
  {'name': 'france24',
   'title': 'France 24',
   'radio': False,
   'logo': 'logos/default.png',
   'modified': 1726160428},
  {'name': 'france-inter',
   'title': 'France Inter',
   'radio': True,
   'logo': 'logos/default.png',
   'modified': 1726160428},
  {'name': 'm6',
   'title': 'M6',
   'radio': False,
   'logo': 'logos/default.png',
   'modified': 1726160428},
  {'name': 'rmc',
   'title': 'RMC',
   'radio': True,
   'logo': 'logos/default.p

## Get EPG

In [62]:
import requests

def get_epg_data(channel, start_gte, start_lte, token,**kwargs):
    url = "https://keywords.mediatree.fr/api/epg/"
    params = {
        "channel": channel,
        "start_gte": start_gte,
        "start_lte": start_lte,
        "token": token,
        **kwargs
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()  # Return the data as JSON if successful
        data = pd.DataFrame(data["data"])
        data['start'] = pd.to_datetime(data['start'], unit='s')  # Convert Unix timestamp in seconds to datetime
        data['stop'] = pd.to_datetime(data['stop'], unit='s')    # Convert Unix timestamp in seconds to datetime
        return data
    else:
        return {"error": f"Request failed with status code {response.status_code}"}

# Example usage
channel = channel_cnews
start_gte = "2024-09-01T00:00:00"
start_lte = "2024-09-01T23:59:59"
token = MEDIATREE_TOKEN

data = get_epg_data(channel, start_gte, start_lte, token)
data

,provider,start,stop,channel,title,sub-title,desc,category,rating,credits
0,{'source-info-url': 'https://github.com/racaca...,2024-09-01 00:03:00,2024-09-01 00:04:00,itele,"{'fr': 'Météo', 'default': 'Météo'}","{'fr': 'Prévisions pour le lendemain', 'defaul...",{'fr': 'Les prévisions météo pour le lendemain...,"{'fr': 'Météo', 'default': 'Météo'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
1,{'source-info-url': 'https://github.com/racaca...,2024-09-01 00:04:00,2024-09-01 00:28:00,itele,"{'fr': 'Edition de la nuit', 'default': 'Editi...",NaN,{'fr': 'Un journal télévisé complet qui revien...,"{'fr': 'Journal', 'default': 'Journal'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
2,{'source-info-url': 'https://github.com/racaca...,2024-09-01 00:28:00,2024-09-01 00:30:00,itele,"{'fr': 'Météo', 'default': 'Météo'}","{'fr': 'Prévisions pour le lendemain', 'defaul...",{'fr': 'Les prévisions météo pour le lendemain...,"{'fr': 'Météo', 'default': 'Météo'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
3,{'source-info-url': 'https://github.com/racaca...,2024-09-01 00:30:00,2024-09-01 00:53:00,itele,"{'fr': 'Edition de la nuit', 'default': 'Editi...",NaN,{'fr': 'Un journal télévisé complet qui revien...,"{'fr': 'Journal', 'default': 'Journal'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
4,{'source-info-url': 'https://github.com/racaca...,2024-09-01 00:53:00,2024-09-01 01:00:00,itele,"{'fr': 'L'heure des livres', 'default': 'L'heu...","{'fr': 'Guillaume Musso', 'default': 'Guillaum...",{'fr': 'Anne Fulda reçoit Guillaume Musso pour...,"{'fr': 'Culture', 'default': 'Culture'}","{'system': 'CSA', 'value': 'Tout public'}",{'guest': 'Anne Fulda'}
5,{'source-info-url': 'https://github.com/racaca...,2024-09-01 01:00:00,2024-09-01 01:01:00,itele,"{'fr': 'Météo', 'default': 'Météo'}","{'fr': 'Prévisions pour le lendemain', 'defaul...",{'fr': 'Les prévisions météo pour le lendemain...,"{'fr': 'Météo', 'default': 'Météo'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
6,{'source-info-url': 'https://github.com/racaca...,2024-09-01 01:01:00,2024-09-01 01:23:00,itele,"{'fr': 'Edition de la nuit', 'default': 'Editi...",NaN,{'fr': 'Un journal télévisé complet qui revien...,"{'fr': 'Journal', 'default': 'Journal'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
7,{'source-info-url': 'https://github.com/racaca...,2024-09-01 01:23:00,2024-09-01 01:31:00,itele,"{'fr': 'L'heure des livres', 'default': 'L'heu...","{'fr': 'Guillaume Musso', 'default': 'Guillaum...",{'fr': 'Anne Fulda reçoit Guillaume Musso pour...,"{'fr': 'Culture', 'default': 'Culture'}","{'system': 'CSA', 'value': 'Tout public'}",{'guest': 'Anne Fulda'}
8,{'source-info-url': 'https://github.com/racaca...,2024-09-01 01:31:00,2024-09-01 01:32:00,itele,"{'fr': 'Météo', 'default': 'Météo'}","{'fr': 'Prévisions pour le lendemain', 'defaul...",{'fr': 'Les prévisions météo pour le lendemain...,"{'fr': 'Météo', 'default': 'Météo'}","{'system': 'CSA', 'value': 'Tout public'}",NaN
9,{'source-info-url': 'https://github.com/racaca...,2024-09-01 01:32:00,2024-09-01 01:56:00,itele,"{'fr': 'Edition de la nuit', 'default': 'Editi...",NaN,{'fr': 'Un journal télévisé complet qui revien...,"{'fr': 'Journal', 'default': 'Journal'}","{'system': 'CSA', 'value': 'Tout public'}",NaN


## Get all subtitles

In [16]:
import requests
import time
import pandas as pd
from tqdm.auto import tqdm
from datetime import datetime, timedelta

def get_subtitles_data(channel, start_gte, start_lte, token, start_results=0, size_results=50, **kwargs):
    url = "https://keywords.mediatree.fr/api/v2/subtitle/"
    params = {
        "channel": channel,
        "start_gte": start_gte,
        "start_lte": start_lte,
        "token": token,
        "from": start_results,
        "size": size_results,
        "type": "s2t",
        **kwargs
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        res = response.json()
        data = pd.DataFrame(res["data"])
        data['start'] = pd.to_datetime(data['start'], unit='s')
        data["channel_name"] = data["channel"].map(lambda x: x["name"])
        data["channel_title"] = data["channel"].map(lambda x: x["title"])
        data["channel_radio"] = data["channel"].map(lambda x: x["radio"])
        data = data.drop(columns=["srt", "channel"])
        return res, data
    else:
        return {"error": f"Request failed with status code {response.status_code}", "response": response}, None

def fetch_all_subtitles(channel, start_gte, start_lte, token, size_results=50):
    initial_res, data = get_subtitles_data(channel, start_gte, start_lte, token, size_results=size_results)
    
    if 'error' in initial_res:
        return initial_res, None
    
    total_results = initial_res["total_results"]
    number_pages = initial_res['number_pages']
    
    print(f"Total results: {total_results}")
    print(f"Number of pages: {number_pages}")
    
    all_data = [data]
    
    for page in range(1, number_pages):
    # for page in tqdm(range(1, number_pages), desc="Fetching pages", leave=False):
        
        start_results = page * size_results
        res, data = get_subtitles_data(channel, start_gte, start_lte, token, start_results, size_results)
        
        if data is not None:
            data["page"] = page
            all_data.append(data)
        else:
            print(f"Error fetching page {page + 1}: {res.get('error', 'Unknown error')}")
    
    if all_data:
        combined_data = pd.concat(all_data, ignore_index=True)
        return {"total_results": total_results, "number_pages": number_pages}, combined_data
    else:
        return {"error": "No data was successfully retrieved"}, None

def fetch_subtitles_by_day(channel, start_date, end_date, token, size_results=50):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    all_data = []
    data_dir = "../data/mediatree/"
    os.makedirs(data_dir, exist_ok=True)

    for current_date in tqdm(date_range, desc="Fetching data by day"):
        next_date = current_date + timedelta(days=1)
        start_gte = current_date.strftime("%Y-%m-%dT%H:%M:%S")
        start_lte = next_date.strftime("%Y-%m-%dT%H:%M:%S")
        
        file_name = f"{data_dir}{channel}_{current_date.strftime('%Y-%m-%d')}.parquet"
        
        if os.path.exists(file_name):
            print(f"Loading existing data for {start_gte} to {start_lte}")
            data = pd.read_parquet(file_name)
            all_data.append(data)
        else:
            print(f"Fetching data for {start_gte} to {start_lte}")
            res, data = fetch_all_subtitles(channel, start_gte, start_lte, token, size_results)
            
            if data is not None:
                data.to_parquet(file_name, index=False)
                all_data.append(data)
            else:
                print(f"Error fetching data for {start_gte} to {start_lte}: {res.get('error', 'Unknown error')}")
            
            # time.sleep(5)  # Add a small delay between days to avoid overwhelming the API

    if all_data:
        combined_data = pd.concat(all_data, ignore_index=True)
        return combined_data
    else:
        return None

# Example usage
channel = channel_cnews
start_date = "2023-09-13T00:00:00"
end_date = "2024-09-12T00:00:00"
token = MEDIATREE_TOKEN

In [17]:
res,data = get_subtitles_data(channel, start_date, end_date, token,start_results = 0,size_results = 10)
data

,start,plaintext,channel_name,channel_title,channel_radio
0,2023-09-13 06:02:00,avec un pronostic vital plus que engagé l' inc...,itele,i>Télé,False
1,2023-09-30 07:44:00,fois on a des images qui nous rappelle marcel ...,itele,CNEWS,False
2,2023-09-30 13:34:00,pour cent du pib en dessous de trois pourcent ...,itele,CNEWS,False
3,2023-09-30 15:46:00,alors c' est à l' issue d' une guerre qui s' e...,itele,CNEWS,False
4,2023-09-30 16:08:00,sur la sur la la les les condamnations des con...,itele,CNEWS,False
5,2023-09-30 12:58:00,assureur d' intérêt général c' était votre pro...,itele,CNEWS,False
6,2023-09-23 18:58:00,pour rouen vingt-deux pour la pointe bretonne ...,itele,i>Télé,False
7,2023-09-24 08:28:00,stations de ski qu' on voit aujourd' hui en po...,itele,i>Télé,False
8,2023-09-24 11:20:00,sont pas forcément distinguer deviennent jean-...,itele,i>Télé,False
9,2023-09-24 14:12:00,hors de la gâchette si vous voulez pour eux d'...,itele,i>Télé,False


In [18]:
result = fetch_subtitles_by_day(channel, start_date, end_date, token,size_results = 1000)
if result is not None:
    print(f"Total rows fetched: {len(result)}")
    print(result.head())
else:
    print("Failed to fetch data")

Fetching data by day:   0%|          | 0/366 [00:00<?, ?it/s]

Fetching data for 2023-09-13T00:00:00 to 2023-09-14T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-14T00:00:00 to 2023-09-15T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-15T00:00:00 to 2023-09-16T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-16T00:00:00 to 2023-09-17T00:00:00
Total results: 544
Number of pages: 1
Fetching data for 2023-09-17T00:00:00 to 2023-09-18T00:00:00
Total results: 544
Number of pages: 1
Fetching data for 2023-09-18T00:00:00 to 2023-09-19T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-19T00:00:00 to 2023-09-20T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-20T00:00:00 to 2023-09-21T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-21T00:00:00 to 2023-09-22T00:00:00
Total results: 574
Number of pages: 1
Fetching data for 2023-09-22T00:00:00 to 2023-09-23T00:00:00
Total results: 574
Number of pages: 1
Fetching d

In [19]:
channel = channel_sudradio
start_date = "2023-09-13T00:00:00"
end_date = "2024-09-12T00:00:00"
token = MEDIATREE_TOKEN

result = fetch_subtitles_by_day(channel, start_date, end_date, token,size_results = 1000)
if result is not None:
    print(f"Total rows fetched: {len(result)}")
    print(result.head())
else:
    print("Failed to fetch data")

Fetching data by day:   0%|          | 0/366 [00:00<?, ?it/s]

Loading existing data for 2023-09-13T00:00:00 to 2023-09-14T00:00:00
Fetching data for 2023-09-14T00:00:00 to 2023-09-15T00:00:00
Total results: 383
Number of pages: 1
Fetching data for 2023-09-15T00:00:00 to 2023-09-16T00:00:00
Total results: 441
Number of pages: 1
Fetching data for 2023-09-16T00:00:00 to 2023-09-17T00:00:00
Total results: 454
Number of pages: 1
Fetching data for 2023-09-17T00:00:00 to 2023-09-18T00:00:00
Total results: 443
Number of pages: 1
Fetching data for 2023-09-18T00:00:00 to 2023-09-19T00:00:00
Total results: 383
Number of pages: 1
Fetching data for 2023-09-19T00:00:00 to 2023-09-20T00:00:00
Total results: 383
Number of pages: 1
Fetching data for 2023-09-20T00:00:00 to 2023-09-21T00:00:00
Total results: 383
Number of pages: 1
Fetching data for 2023-09-21T00:00:00 to 2023-09-22T00:00:00
Total results: 383
Number of pages: 1
Fetching data for 2023-09-22T00:00:00 to 2023-09-23T00:00:00
Total results: 441
Number of pages: 1
Fetching data for 2023-09-23T00:00:00 to

In [193]:
51 * 200

10200

In [209]:
res,data = get_subtitles_data(channel, start_gte, start_lte, token,start_results = 0,size_results = 1000)
data

In [211]:
res["response"].json()

{'error': 'Unauthorized', 'message': 'Token expired'}

In [183]:
res,data = get_subtitles_data(channel, start_gte, start_lte, token,start_results = 1,size_results = 300)
data

,start,plaintext,channel_name,channel_title,channel_radio
0,2024-06-01 04:54:00,de question de prendre le risque de gâcher ça ...,itele,CNEWS,False
1,2024-06-01 05:08:00,pierres et le caractère spectaculaire mais aus...,itele,CNEWS,False
2,2024-06-01 05:24:00,cents quatre-vingt neuf euros la baignade esti...,itele,CNEWS,False
3,2024-06-01 05:28:00,trois cent vingt-neuf euros ça donne très envi...,itele,CNEWS,False
4,2024-06-01 05:34:00,soit un doublement en l' espace de cinq ans eu...,itele,CNEWS,False
...,...,...,...,...,...
295,2024-06-19 04:42:00,et taxes pour cent électrique gamme utilitaire...,itele,CNEWS,False
296,2024-06-19 04:46:00,bus des procurations stationne devant le commi...,itele,CNEWS,False
297,2024-06-19 04:50:00,aleos thermostat connecté à vos radiateurs pou...,itele,CNEWS,False
298,2024-06-19 05:40:00,à savoir la rapidité de la déliquescence du pa...,itele,CNEWS,False


In [20]:
# %%time
# res,full_data = fetch_all_subtitles(channel, start_gte, start_lte, token,size_results = 200,existing_data = full_data)

# Post-processing

## Reload all files

In [21]:
data_dir = "../data/mediatree"

In [25]:
all_parquet = [os.path.join(data_dir,x) for x in os.listdir(data_dir)]
df = pd.concat([pd.read_parquet(x) for x in tqdm(all_parquet)],ignore_index = True)

  0%|          | 0/732 [00:00<?, ?it/s]

In [28]:
import tiktoken
from nltk.tokenize import wordpunct_tokenize

In [32]:
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

def num_tokens_from_string(encoding,string: str) -> int:
    """Returns the number of tokens in a text string."""
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Compute number of characters and number of words and tokens (approximate following approx conversion number by OpenAI)
df["num_characters"] = df["plaintext"].map(len)
df["num_words"] = df["plaintext"].map(lambda x : len(wordpunct_tokenize(x)))
df["num_tokens"] = df["plaintext"].map(lambda x : num_tokens_from_string(encoding,x))
df["num_tokens_approx"] = (df["num_words"] / 0.75).astype(int)

In [34]:
df["num_tokens"].sum()

168050237

In [36]:
df.to_parquet("../data/mediatree_raw_1year.parquet")

## Repunctuation

In [39]:
from deepmultilingualpunctuation import PunctuationModel

model = PunctuationModel()
text = "My name is Clara and I live in Berkeley California Ist das eine Frage Frau Müller"
result = model.restore_punctuation(text)
print(result)

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/theolvs/miniconda3/lib/python3.12/site-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


My name is Clara and I live in Berkeley, California. Ist das eine Frage, Frau Müller?


In [42]:
text = df.iloc[0]["plaintext"]
result = model.restore_punctuation(text)
result

"avec un pronostic vital plus que engagé. l' incendie dans une décharge sauvage de nemours en seine-et-marne est toujours actif. cette décharge est alimenté par des membres de la communauté des gens du voyage installé juste à côté. la mairie souhaite d' ailleurs l' expulsion de ce camp. rejoint tout de suite adrios peter, ilôt marchegay, sur place, alors adrien, depuis lundi, le feu n' ait toujours pas éteint et les pompiers sont toujours mob. utiliser, ce matin, exactement, sean a d' ailleurs il y chemin a d' de ailleurs, il plus y en a de plus, plus de en plus pompiers, de pompiers. ici, ils sont mobilisées depuis lundi, car le feu est circonscrit, mène toujours pas éteint. leurs ici, on se trouve devant et bien le site de commandement des pompiers. vous le voyez sur ces images de léo marchegay. je vous disais tout, alors qu' un point d' étape devait avoir lieu à sept heures trente. il vient à l' instant de se terminer. les pompiers nous confirme que près de quatre mille cinq cents m

## Climate classification

In [43]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from tqdm.auto import tqdm

# dataset_name = "climatebert/climate_detection"
model_name = "climatebert/distilroberta-base-climate-detector"

# # If you want to use your own data, simply load them as 🤗 Datasets dataset, see https://huggingface.co/docs/datasets/loading
# dataset = datasets.load_dataset(dataset_name, split="test")

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [45]:
pipe(text[:200])

[{'label': 'no', 'score': 0.9799736738204956}]